In [1]:
from UniDec.unidec_modules import mzmlparse_auto as automzml
from UniDec import unidec
from UniDec.unidec_modules import ChromEng as chrom
from UniDec.UniChrom2 import *
from pathlib import Path
import os
import matplotlib.pyplot as plt

ImportError: cannot import name 'multiprocessing' from 'UniDec' (C:\Users\cm19ljc\Documents\GitHub\MS-Analysis\UniDec\__init__.py)

In [ ]:
mzml = "mzML.mzML"
# folder_name = mzml[:-5] + "_upload"
curr_dir =  os.getcwd()
path = curr_dir + "/" + mzml[:-5] + "/" + mzml # (going to make a new folder containing data eventually)
# Path(folder_name).mkdir(parents=True, exist_ok = True) # make folder to store files 
# store_dir =  os.getcwd() + "/" + folder_name # get location of new folder 

# automzml.extract(mzml, curr_dir, 1, "txt")

In [ ]:
eng = chrom.ChromEngine()
eng.open_chrom(path)

In [ ]:
# -- Data processing 
eng.config.subtype = 2 # background subtraction - subtract curved
eng.config.subbuff = 100 # background subtraction amount(subtract curved) 0 = 0ff, 100 = good amount when on
eng.config.datanorm = 0 # turn off data normalisation

# -- Deconvolution
eng.config.numit = 100 # number of iterations

# mass range (default = 5000.0 to 500000.0 Da)
eng.config.massub = 15000 # upper 
eng.config.masslb = 11000 # lower

eng.config.massbins = 0.1 # sample mass every 0.1 Da

# FWHM 
# eng.get_auto_peak_width()
eng.config.mzsig = 0 

# charge range
eng.config.startz = 1
eng.config.endz = 20

# smoothing 
eng.config.zzsig = 1 # charge smooth width (smooth charge state distributions)
eng.config.psig = 1 # smooth nearby points (point smooth width, some = 1)
eng.config.beta = 0 # suppress artifacts (beta, some = 50)

eng.config.psfun = 0 # Peak shape function (gaussian, lorentzian, split G/L)

# Peak Selection and plotting
eng.config.peaknorm = 0 # Normalise peaks (0 = off)
eng.config.peakwindow = 100 # integration window / Da
eng.config.nativeub = 10
eng.config.nativelb = 10


In [ ]:
plt.plot(eng.tic[:, 0], eng.tic[:, 1])
plt.show()

In [ ]:
eng.tic[:, 0]

In [ ]:
plt.plot(eng.ticdat[:, 0]*60, eng.ticdat[:, 1])
plt.xlim(55, 70)
plt.show()

In [ ]:
# create new attributes defining location of peaks in tic
eng.firstpeak = 62
eng.peakspacing = 56
eng.totalpeaks = 3
eng.peakwindowll = 5
eng.peakwindowul = 5

In [ ]:
eng.peaktimes = [eng.firstpeak + i*eng.peakspacing for i in range(eng.totalpeaks)]
eng.peaktimes

In [ ]:
# use this attribute to update peak windows
eng.chrompeaks_tranges = [[(i-eng.peakwindowll)/60, (i+eng.peakwindowul)/60]for i in eng.peaktimes] # get peak windows
eng.chrompeaks_tranges

In [ ]:
# add peak windows into unidec

times = eng.chrompeaks_tranges

for i, t in enumerate(times): # copied from add_chrom_peaks() in ChromEng.py
    data = eng.get_data_from_times(t[0], t[1])
    eng.data.add_data(data, name=str(t[0]), attrs=eng.attrs, export=False)

In [ ]:
plt.plot(eng.mzdata[:, 0], eng.mzdata[:, 1])
plt.show()

In [ ]:
eng.data.spectra # spectra stored in mudstruct (MetaDataSet) along with rest of data

In [ ]:
# taken from UniChrom2 - on_unidec_run

eng.

I don't get what data it's processing - it looks like it's only processing mzdata which seems to be only one spectrum??? 

In [ ]:
fname = os.path.splitext(eng.filename)[0]+"_selection.txt"

eng.unidec_eng.pass_data_in(eng.mzdata, dirname=eng.config.udir, fname=fname)
eng.config.config_export(eng.unidec_eng.config.confname)
eng.unidec_eng.config.config_import(eng.unidec_eng.config.confname)
eng.unidec_eng.process_data()
eng.unidec_eng.run_unidec(efficiency=True)

In [ ]:
eng.massdat = eng.unidec_eng.data.massdat

In [ ]:
# Start again using ChromApp
app = 